In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
import sklearn
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Import libary for TFID Vectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from  sklearn.metrics  import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Import feature selection Libraries
from sklearn.feature_selection import SelectKBest, chi2, f_regression, mutual_info_classif

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import math
%matplotlib inline
import seaborn as sns

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
!wget -O dataset_combined_uncleaned.csv https://raw.githubusercontent.com/godhi06/TA/master/data/dataset_combined_uncleaned.csv
!wget -O df_pre1.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/dataset_preprocessed_1.csv


df_ori = pd.read_csv('dataset_combined_uncleaned.csv', index_col=0)
df_pre1 = pd.read_csv('df_pre1.csv', index_col=0)

--2020-08-03 03:01:13--  https://raw.githubusercontent.com/godhi06/TA/master/data/dataset_combined_uncleaned.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [text/plain]
Saving to: ‘dataset_combined_uncleaned.csv’

dataset_combined_un 100%[===================>] 218.48K  --.-KB/s    in 0.05s   

2020-08-03 03:01:14 (4.25 MB/s) - ‘dataset_combined_uncleaned.csv’ saved [223725/223725]

--2020-08-03 03:01:18--  https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/dataset_preprocessed_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaitin

In [ ]:
df_ori_dep = df_ori[df_ori.label == 1]
df_ori_nonDep = df_ori[df_ori.label == 0]
print("Banyak data original:", len(df_ori))
print("Banyak data depresi:", len(df_ori_dep))
print("Banyak data non-depresi:", len(df_ori_nonDep))

Banyak data original: 657
Banyak data depresi: 340
Banyak data non-depresi: 317


In [ ]:
df_ori = df_ori.drop_duplicates()

In [ ]:
df_ori_dep = df_ori[df_ori.label == 1]
df_ori_nonDep = df_ori[df_ori.label == 0]
print("Banyak data original:", len(df_ori))
print("Banyak data depresi:", len(df_ori_dep))
print("Banyak data non-depresi:", len(df_ori_nonDep))
print('\n')
print("Banyak data depresi dfPre1:", len(df_pre1[df_pre1.label == 1]))
print("Banyak data non-depresi dfPre1:", len(df_pre1[df_pre1.label == 0]))

Banyak data original: 651
Banyak data depresi: 340
Banyak data non-depresi: 311


Banyak data depresi dfPre1: 340
Banyak data non-depresi dfPre1: 307


PREPROCESSING

In [ ]:
def text_to_lowerCase(text):
    return text.lower()

def expand_contractions(data):
    data = re.sub(r"\bdon't\b", 'do not', data)
    data = re.sub(r"\bdidn't\b", 'did not', data)
    data = re.sub(r"\bdoesn't\b", 'does not', data)
    data = re.sub(r"\bisn't\b", 'is not', data)
    data = re.sub(r"\baren't\b", 'are not', data)
    data = re.sub(r"\bwasn't\b", 'was not', data)
    data = re.sub(r"\bweren't\b", 'were not', data)

    data = re.sub(r"\bhadn't\b", 'had not', data)
    data = re.sub(r"\bhadn't've\b", 'had not have', data)
    data = re.sub(r"\bhasn't\b", 'has not', data)
    data = re.sub(r"\bhaven't\b", 'have not', data)

    data = re.sub(r"\bcan't\b", 'can not', data)
    data = re.sub(r"\bcan't've\b", 'cannot have', data)
    data = re.sub(r"\bcould've\b", 'could have', data)
    data = re.sub(r"\bcouldn't\b", 'could not', data)
    data = re.sub(r"\bcouldn't've\b", 'could not have', data)
    data = re.sub(r"\bshould've\b", 'should have', data)
    data = re.sub(r"\bshouldn't\b", 'should not', data)
    data = re.sub(r"\bshouldn't've\b", 'should not have', data)

    data = re.sub(r"\bi'll\b", 'i will', data)
    data = re.sub(r"\bi'll've\b", 'i will have', data)
    data = re.sub(r"\bi'm\b", 'i am', data)
    data = re.sub(r"\bi've\b", 'i have', data)
    data = re.sub(r"\bi'd\b", 'i would', data)
    data = re.sub(r"\bi'd've\b", 'i would have', data)

    data = re.sub(r"\by'all\b", 'you all', data)
    data = re.sub(r"\by'all're\b", 'you all are', data)
    data = re.sub(r"\byou're\b", 'you are', data)
    data = re.sub(r"\byou've\b", 'you have', data)
    data = re.sub(r"\byou'll\b", 'you will', data)
    data = re.sub(r"\byou'll've\b", 'you will have', data)
    data = re.sub(r"\byou'd\b", 'you would', data)
    data = re.sub(r"\byou'd've\b", 'you would have', data)

    data = re.sub(r"\bwe're\b", 'we re', data)
    data = re.sub(r"\bwe've\b", 'we have', data)
    data = re.sub(r"\bwe'll\b", 'we will', data)
    data = re.sub(r"\bwe'll've\b", 'we will have', data)
    data = re.sub(r"\bwe'd\b", 'we would', data)
    data = re.sub(r"\bwe'd've\b", 'we would have', data)

    data = re.sub(r"\bthey're\b", 'they are', data)
    data = re.sub(r"\bthey've\b", 'they have', data)
    data = re.sub(r"\bthey'll\b", 'they will', data)
    data = re.sub(r"\bthey'll've\b", 'they will have', data)
    data = re.sub(r"\bthey'd\b", 'they would', data)
    data = re.sub(r"\bthey'd've\b", 'they would have', data)

    data = re.sub(r"\bhe's\b", 'he is', data)
    data = re.sub(r"\bhe'd\b", 'he would', data)
    data = re.sub(r"\bhe'd've\b", 'he would have', data)
    data = re.sub(r"\bhe'll\b", 'he will', data)
    data = re.sub(r"\bhe'll've\b", 'he will have', data)

    data = re.sub(r"\bshe's\b", 'she is', data)
    data = re.sub(r"\bshe'd\b", 'she would', data)
    data = re.sub(r"\bshe'd've\b", 'she would have', data)
    data = re.sub(r"\bshe'll\b", 'she will', data)
    data = re.sub(r"\bshe'll've\b", 'she will have', data)

    data = re.sub(r"\bit's\b", 'it is', data)
    data = re.sub(r"\bit'd\b", 'it would', data)
    data = re.sub(r"\bit'd've\b", 'it would have', data)
    data = re.sub(r"\bit'll\b", 'it will', data)

    data = re.sub(r"\bthat's\b", 'that is', data)
    data = re.sub(r"\bthat'd\b", 'that would', data)
    data = re.sub(r"\bthat'd've\b", 'that would have', data)

    data = re.sub(r"\bthere's\b", 'there is', data)
    data = re.sub(r"\bthere'd\b", 'there would', data)
    data = re.sub(r"\bthere'd've\b", 'there would have', data)

    data = re.sub(r"\bwhat's\b", 'what is', data)
    data = re.sub(r"\bwhat're\b", 'what are', data)
    data = re.sub(r"\bwhat'd\b", 'what would', data)
    data = re.sub(r"\bwhat've\b", 'what have', data)

    data = re.sub(r"\bwhat's\b", 'when is', data)
    data = re.sub(r"\bwhat're\b", 'where is', data)
    data = re.sub(r"\bwhat'd\b", 'who is', data)
    data = re.sub(r"\bwhat've\b", 'who will', data)

    data = re.sub(r"\bwill've\b", 'will have', data)
    data = re.sub(r"\bwon't\b", 'will not', data)
    data = re.sub(r"\bwould've\b", 'would have', data)
    data = re.sub(r"\bwouldn't\b", 'would not', data)

    data = re.sub(r"\bhow'd\b", 'how did', data)
    data = re.sub(r"\bhow'd'yb", 'how do you', data)
    data = re.sub(r"\bhow'll\b", 'how will', data)
    data = re.sub(r"\bhow's\b", 'how is', data)

    data = re.sub(r"\bmight've\b", 'might have', data)
    data = re.sub(r"\bmightn't\b", 'might not', data)
    data = re.sub(r"\bmust've\b", 'must have', data)
    data = re.sub(r"\bmustn't\b", 'must not', data)
    data = re.sub(r"\bneedn't\b", 'need not', data)

    data = re.sub(r"\b'cause\b", 'because', data)
    data = re.sub(r"\blet's\b", 'let us', data)
    data = re.sub(r"\bo'clock\b", 'of the clock', data)

    data = data.split()
    data = " ".join(data)
    return data

# Remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Remove numbers
def remove_numbers(text):
    text = re.sub('\d+', '', text)
    return text

# Remove special characters
def remove_special_characters(text):
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    return text

# Remove extra whitespaces
def remove_extraspaces(text):
    return re.sub(' +', ' ', text)

def tokenized_tweet(text):
    return text.split()

def cleaning(text):
    text = text_to_lowerCase(text)
    text = expand_contractions(text)
    text = remove_urls(text)
    text = remove_numbers(text)
    text = remove_special_characters(text)
    text = remove_extraspaces(text)
    return text

In [ ]:
# Clean data using cleaning function
df_ori_pre1 = df_ori.copy()
df_ori_pre1['text'] = df_ori.text.apply(cleaning)

# drop duplicate
df_ori = df_ori.drop_duplicates()
df_ori_pre1 = df_ori_pre1.drop_duplicates()

In [ ]:
# Copy dataframe untuk buat dataframe tanpa stopwords
df_ori_pre2 = df_ori_pre1.copy()

In [ ]:
df_ori_pre2_tok = df_ori_pre2.text.apply(lambda x: x.split())
df_ori_pre2_tok = df_ori_pre2_tok.apply(lambda x: [w for w in x if w not in list(stop_words)])

In [ ]:
df_ori_pre2['text'] = list(df_ori_pre2_tok.apply(lambda x: ' '.join(x)))
df_ori_pre2['wc_count'] = df_ori_pre2.text.apply(lambda x: len(str(x).split()))

In [ ]:
idx_all = df_ori_pre2[df_ori_pre2.wc_count >= 3].index

In [ ]:
# Sesuaikan semua dataFrame dengan idx_all 
df_ori = df_ori.loc[idx_all]
df_ori_pre1 = df_ori_pre1.loc[idx_all]
df_ori_pre2 = df_ori_pre2.loc[idx_all]

In [ ]:
len(df_ori) == len(df_ori_pre1) == len(df_ori_pre2)

True

In [ ]:
df_ori.reset_index(drop=True, inplace=True)
df_ori_pre1.reset_index(drop=True, inplace=True)
df_ori_pre2.reset_index(drop=True, inplace=True)

In [ ]:
from google.colab import files #library untuk export dari googlecolab

df_ori.to_csv('df_ori.csv')

df_ori_pre1.to_csv('df_ori_pre1.csv')

df_ori_pre2.to_csv('df_ori_pre2.csv')

files.download('df_ori.csv')
files.download('df_ori_pre1.csv')
files.download('df_ori_pre2.csv')

create data with stemming and lemmatization

In [ ]:
stemmer = PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [ ]:
def stem_lemm(df):
    pre_tok = df.text.apply(lambda x: x.split())
    pre_tok_stem = pre_tok.apply(lambda x: [stemmer.stem(i) for i in x])
    pre_tok_lemm = pre_tok.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
    
    # Join tokens into dataframe
    df_pre_stem = pd.DataFrame(pre_tok_stem.apply(lambda x: ' '.join(x)))
    df_pre_lemm = pd.DataFrame(pre_tok_lemm.apply(lambda x: ' '.join(x)))

    # Add labels into each dataframe
    df_pre_stem['label'] = list(df.label)
    df_pre_lemm['label'] = list(df.label)
    
    return [df_pre_stem, df_pre_lemm]

In [ ]:
pre1_group = stem_lemm(df_ori_pre1)
pre2_group = stem_lemm(df_ori_pre2)

In [ ]:
from google.colab import files #library untuk export dari googlecolab

pre1_group[0].to_csv('pre1_stemmed.csv')

pre1_group[1].to_csv('pre1_lemma.csv')

pre2_group[0].to_csv('pre2_stemmed.csv')

pre2_group[1].to_csv('pre2_lemma.csv')

files.download('pre1_stemmed.csv')
files.download('pre1_lemma.csv')
files.download('pre2_stemmed.csv')
files.download('pre2_lemma.csv')